# IST256 Project Deliverable 3 (P3)

## Phase 3: Data Story / Coding for Explanation

In this step, you submit the final version of your working code. You should be implementing the data story that you discussed in P2 (2.3.1). 

All code necessary to make the project run should be included in this notebook. This include all imports, functions, setup code and your interact. There should be no code that causes errors or exploratory code here.

The expectation is that your instructor can open this notebook, run all cells, and then use your program.

The code you write should be clear, easy to understand and use the affordances learned in the course.

No changes to your code will be considered after this submission. It is important to take prior instructor feedback taken into consideration and these factor into your evaluation.


### Step 1: Summarize Enhancements and Changes

If there were any enhancement or changes to your P3 from your P2 (including those you suggested), please explain them here. For example you might have geocoded your dataset or extracted entities from the text.


- I want to add more information to all of the markers on the map.
-I want to add the potential use of interact manual to make the software more intuitive.
-I want to try to make two more graphs. (most of my data are strings so its very hard to graph and chart them)
-I want to add a count column
-I want to see if there are anymore ways to add interger replated data

### Step 2: Project Code

Include all project code below. This includes code that enhances the original dataset. Make sure to execute your code to ensure it runs properly before you turn it in. 

Add as many cells as you need here.


In [1]:
# 2.1 Code here
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Image
from ipywidgets import interact_manual
from math import radians, sin, cos, sqrt, atan2
from ipywidgets import interact_manual, HTML
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from time import sleep
import seaborn as sns
import folium
import warnings
warnings.filterwarnings('ignore')

# Load the EV charging stations dataset
ev_data = pd.read_csv('Electric_Vehicle_Charging_Stations.csv')
ev_data.sample(10)

,Station Name,Street Address,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,New Georeferenced Column
41,City of New Haven - Sherman Tyler Lot,60 N Frontage Rd,New Haven,24 hours daily; permit lot,NONE,2,NONE,NONE,POINT (-72.948514 41.306474)
266,Wallingford Shopping Center,930 N Colony Rd,Wallingford,24 hours daily,NONE,2,NONE,NONE,POINT (-72.807124 41.483774)
233,CT RAIL,51 Depot Rd,Berlin,24 hours daily,NONE,2,NONE,NONE,POINT (-72.7653384 41.6356154)
244,Cumberland Farms,471 New Park Ave,West Hartford,24 hours daily,NONE,NONE,1,NONE,POINT (-72.718112 41.741696)
149,Town of Groton - Town Hall,45 Fort Hill Rd,Groton,24 hours daily,NONE,2,NONE,NONE,POINT (-72.033489 41.345816)
289,Butler Street Parking Lot,30 Butler St,Meriden,24 hours daily,NONE,2,NONE,NONE,POINT (-72.805988 41.53697)
327,Delamar West Hartford - Tesla Destination,1 Memorial Rd,West Hartford,24 hours daily; for customer use only,NONE,3,NONE,NONE,POINT (-72.737672 41.760566)
183,Hoffman Nissan,46 Albany Turnpike,West Simsbury,Dealership business hours,NONE,1,NONE,NONE,POINT (-72.869319 41.820371)
146,Rocky Neck State Park,244 W Main St,Niantic,Dawn to dusk,NONE,2,NONE,NONE,POINT (-72.24323 41.300883)
313,Park 'N Fly - Bradley International Airport,1186 South St,Suffield,24 hours daily; pay lot; see attendant for access,NONE,2,NONE,NONE,POINT (-72.663078 41.951328)


In [2]:
# Assuming your data is in a CSV file named 'ev_data.csv'
def process_ev_data(filename):
  """
  This function reads an EV charging station data CSV file, creates a new column
  indicating charger levels, and saves the modified data to a new CSV file.

  Args:
      filename (str): The name of the CSV file containing EV charging station data.
  """
  # Read the CSV data
  ev_data = pd.read_csv("Electric_Vehicle_Charging_Stations.csv")

  # Define a function to create level strings
  def get_level_string(num):
    if num == 'NONE':
      return 'NONE'
    elif num == 1:
      return 'Level 1'
    elif num == 2:
      return 'Level 2'
    else:
      return f'Level {num}'  # Handle unexpected values

  # Create new column with level strings for EV Level1, Level2, and DC Fast
  ev_data['Level 1'] = ev_data['EV Level1 EVSE Num'].apply(get_level_string)
  ev_data['Level 2'] = ev_data['EV Level2 EVSE Num'].apply(get_level_string)
  ev_data['DC Fast'] = ev_data['EV DC Fast Count'].apply(get_level_string)

  # Combine level information into a single column (optional)
  ev_data['Charger Levels'] = ev_data[['Level 1', 'Level 2', 'DC Fast']].apply(lambda x: ', '.join(x.dropna()), axis=1)  # Combine non-empty values

  # Save the modified data to a new CSV file (adjust filename as needed)
  ev_data.to_csv('ev_data_with_levels.csv', index=False)

# Call the function to process the data
process_ev_data('Electric_Vehicle_Charging_Stations.csv')

print("Data processed and saved to 'ev_data_with_levels.csv'")

Data processed and saved to 'ev_data_with_levels.csv'


In [3]:
ev_data_new = pd.read_csv('ev_data_with_levels.csv')
ev_data_new.sample(10)

,Station Name,Street Address,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,New Georeferenced Column,Level 1,Level 2,DC Fast,Charger Levels
10,Brass Mill Center - Shoppers World,495 Union St,Waterbury,6am-12am daily,NONE,2,NONE,NONE,POINT (-73.025659 41.550922),NONE,Level 2,NONE,"NONE, Level 2, NONE"
363,MYSTIC AQUARIUM,55 Coogan Blvd,Mystic,24 hours daily,NONE,NONE,1,NONE,POINT (-71.954088 41.373074),NONE,NONE,Level 1,"NONE, NONE, Level 1"
146,Rocky Neck State Park,244 W Main St,Niantic,Dawn to dusk,NONE,2,NONE,NONE,POINT (-72.24323 41.300883),NONE,Level 2,NONE,"NONE, Level 2, NONE"
153,Wesleyan University,56 Hamlin Ct,MIddletown,24 hours daily,2,NONE,NONE,NONE,POINT (-72.65242 41.557262),Level 2,NONE,NONE,"Level 2, NONE, NONE"
100,Windham High School,355 High St,Windham,24 hours daily,NONE,2,NONE,NONE,POINT (-72.214432 41.723169),NONE,Level 2,NONE,"NONE, Level 2, NONE"
87,Madison Beach Hotel - Tesla Destination,94 W Wharf Rd,Madison,24 hours daily; overnight valet fee,NONE,2,NONE,NONE,POINT (-72.608573 41.27077),NONE,Level 2,NONE,"NONE, Level 2, NONE"
283,Milford Travel Plaza - Tesla Supercharger,I-95 Milford,Milford,24 hours daily; for Tesla use only,NONE,NONE,2,NONE,POINT (-73.010522 41.246242),NONE,NONE,Level 2,"NONE, NONE, Level 2"
236,Woodbridge Town Hall,11 Meetinghouse Ln,Woodbridge,24 hours daily,NONE,2,NONE,NONE,POINT (-73.014176 41.354138),NONE,Level 2,NONE,"NONE, Level 2, NONE"
315,REYNOLDS SUBARU,272 Hamburg Rd,Lyme,24 hours daily,NONE,2,NONE,NONE,POINT (-72.3518219 41.3890729),NONE,Level 2,NONE,"NONE, Level 2, NONE"
261,Wesleyan University,161 Cross St,Middletown,24 hours daily,NONE,4,NONE,NONE,POINT (-72.662214 41.550883),NONE,Level 4,NONE,"NONE, Level 4, NONE"


In [4]:

# First, drop rows with missing georeferenced information
ev_data_new.dropna(subset=['New Georeferenced Column'], inplace=True)

# Extract latitude and longitude into new columns in the DataFrame
ev_data_new[['Latitude', 'Longitude']] = ev_data_new['New Georeferenced Column'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)

# Save the DataFrame to a new CSV file
ev_data_new.to_csv('ev_data_with_coords.csv', index=False)

In [5]:
print(ev_data_new.head())

                          Station Name          Street Address          City  \
0                        BMW OF DARIEN        138-142 Ledge Rd        Darien   
1         Dunkin’ - Tesla Supercharger           893 E Main St       Meriden   
2  Town of Beacon Falls - Commuter Lot           105 N Main St  Beacon Falls   
3                      OLD SAYBROOK VW  319 Middlesex Turnpike  Old Saybrook   
4               Fairfield Rail Station        80 Mill Plain Rd     Fairfield   

                     Access Days Time EV Level1 EVSE Num EV Level2 EVSE Num  \
0                      24 hours daily               NONE                  2   
1  24 hours daily; for Tesla use only               NONE               NONE   
2                      24 hours daily               NONE                  1   
3                      24 hours daily               NONE                  2   
4                      24 hours daily               NONE                  2   

  EV DC Fast Count EV Other Info            

In [6]:


# Load the CSV file with georeferenced column
ev_data_new = pd.read_csv('ev_data_with_coords.csv')

# Drop rows with missing georeferenced information
ev_data_new.dropna(subset=['New Georeferenced Column'], inplace=True)

# Create a new DataFrame for location extraction
location_df = ev_data_new['New Georeferenced Column'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').rename(columns={0: 'Longitude', 1: 'Latitude'}).astype(float)

# Define the center of the map
map_center = [41.6032, -73.0877]  # Coordinates of Connecticut

# Create a folium map centered around Connecticut
ev_map = folium.Map(location=map_center, zoom_start=9.2)

# Add markers to the map with popup information
for index, row in location_df.iterrows():
    location = [row['Latitude'], row['Longitude']]
    # Get corresponding information from the original DataFrame
    ev_info = ev_data_new.iloc[index]
    popup_text = f"Station Name: {ev_info['Station Name']}<br>Street Address: {ev_info['Street Address']}<br>City: {ev_info['City']}"
    folium.Marker(location=location, popup=popup_text).add_to(ev_map)

# Function to calculate distance between two points using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # approximate radius of Earth in kilometers

    # Convert latitude and longitude from degrees to radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # Calculate the change in coordinates
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance
    distance = R * c
    return distance

# Function to handle marker placement based on user input
@interact_manual(latitude=(min(ev_data_new['Latitude']), max(ev_data_new['Latitude'])), longitude=(min(ev_data_new['Longitude']), max(ev_data_new['Longitude'])))
def find_closest_charger(latitude, longitude):
    coords_clicked = [latitude, longitude]
    distances = [haversine(latitude, longitude, row['Latitude'], row['Longitude']) for _, row in ev_data_new.iterrows()]
    closest_station_index = distances.index(min(distances))
    closest_station = ev_data_new.iloc[closest_station_index]
    closest_station_location = [closest_station['Latitude'], closest_station['Longitude']]
    closest_station_popup = f"<b>Closest Station Name:</b> {closest_station['Station Name']}<br><b>Street Address:</b> {closest_station['Street Address']}<br><b>City:</b> {closest_station['City']}"
    folium.Marker(location=closest_station_location, popup=closest_station_popup, icon=folium.Icon(color='red')).add_to(ev_map)
    
    # Add HTML widget to display closest station info
    closest_station_info = HTML(value=f"<h4>Closest Station Information:</h4>{closest_station_popup}")
    display(closest_station_info)

# Display the map
ev_map


interactive(children=(FloatSlider(value=-72.8024843, description='latitude', max=-71.88232300000001, min=-73.7…

### Prepare for your Pitch and Reflection (P4)

With the project code complete, its time to prepare for the final deliverable - submitting your project demo Pitch and reflection.


In [ ]:
# run this code to turn in your work!
from casstools.assignment import Assignment
Assignment().submit()

✅ TIMESTAMP  : 2024-05-06 12:38
✅ COURSE     : ist256
✅ TERM       : spring2024
✅ USER       : abparekh@syr.edu
✅ STUDENT    : True
✅ PATH       : ist256/spring2024/lessons/project/P3.ipynb
✅ ASSIGNMENT : P3.ipynb
✅ POINTS     : 0
✅ DUE DATE   : 2024-05-07 23:59
✅ LATE       : False
✅ STATUS     : New Submission



❓ Submit? [y/n] ❓  y
